<a href="https://colab.research.google.com/github/bukits/dog-breed-identification/blob/main/dog_breed_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np

In [10]:
data_dir = '/content/drive/MyDrive/Images'

labels = [ item for item in sorted(os.listdir(data_dir)) if
           os.path.isdir(os.path.join(data_dir, item)) ]
           
label_dir_paths = []
for l in labels:
    label_dir_paths.append(os.path.join(data_dir, l))

In [11]:
X_paths = []
Y = []
for label_dir in label_dir_paths:
    for img in os.scandir(label_dir):
        X_paths.append(img.path)
        breed_name = label_dir.split("-")[-1]
        Y.append(breed_name)
print('Number of the images:', len(X_paths))

Number of the images: 20598


In [ ]:
X = []

IMG_SIZE = 64

def load_and_resize(image_filename):
  img = cv2.imread(image_filename, cv2.IMREAD_COLOR)
  X.append(cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC))
  print(len(X))

for name in X_paths:
  load_and_resize(name)

X = np.array(X)
Y = np.array(Y)

In [30]:
print('Shape of image is: {}' .format(X.shape))
print('Shape of labels is: {}' .format(Y.shape))

Shape of image is: (20598, 64, 64, 3)
Shape of labels is: (120,)
